# 图的表示方式

## 1. 邻接表

In [1]:
A,B,C,D,E,F,G,H,I = range(9)
graph = {
    A: [B, F],
    B: [C, I, G],
    C: [B, I, D],
    D: [C, I, G, H, E],
    E: [D, H, F],
    F: [A, G, E],
    G: [B, F, H, D],
    H: [G, D, E],
    I: [B, C, D],
}

## 2. 邻接矩阵(Adjacency Matrix)

In [2]:
A,B,C,D,E,F,G,H,I = range(9)
INF = float('inf')
AM = [[INF]*9 for i in range(9)]
for key,values in graph.items():
    for i in values:
        AM[key][i] = 1
for i in range(9):
    AM[i][i] = 0

In [3]:
AM

[[0, 1, inf, inf, inf, 1, inf, inf, inf],
 [inf, 0, 1, inf, inf, inf, 1, inf, 1],
 [inf, 1, 0, 1, inf, inf, inf, inf, 1],
 [inf, inf, 1, 0, 1, inf, 1, 1, 1],
 [inf, inf, inf, 1, 0, 1, inf, 1, inf],
 [1, inf, inf, inf, 1, 0, 1, inf, inf],
 [inf, 1, inf, 1, inf, 1, 0, 1, inf],
 [inf, inf, inf, 1, 1, inf, 1, 0, inf],
 [inf, 1, 1, 1, inf, inf, inf, inf, 0]]

# 图的遍历方式

## 1. 广度优先搜索（Bredth_First_Search）_BFS

In [19]:
# 利用队列实现
# 从起始结点开始依次进入队列，然后弹出
# 每次弹出一个结点，就把该结点没有进过队列的邻结点放入队列，直到队列弹出
from queue import Queue
def BFS(graph, first_node):
    if first_node == None:return  # 图为空，直接返回
    nodes_searched = set()  # nodes_searched : 集合，存放已遍历过的图。
    queue = Queue()
    nodes_searched.add(first_node)  # 将起始结点加入集合
    queue.put(first_node)  # 将起始结点加入队列，使用put方法
    while not queue.empty():
        cur = queue.get()  # 队列非空，则弹出队首元素，使用get方法获取弹出元素值
        print(cur, end=' ')  # 对当前元素进行操作（打印，可为其他，如赋值...）
        for combine_node in graph[cur]:  # combine_node : 弹出结点的相邻结点
            if combine_node not in nodes_searched:  # 如果相邻结点不在nodes_search中
                nodes_searched.add(combine_node)  # 则加入
                queue.put(combine_node)  # 并加入队列

## 2. 深度优先搜索（Depth_First_Search）_DFS

In [5]:
'''
递归版本
'''
# 从起始结点开始不断访问邻居结点，如果访问过，则退回，换另一条路径访问
nodes_searched = set()  # 为了使每次递归时不清空nodes_searched，需写在循环外面
def DFS_recursive(graph, start_node):
    nodes_searched.add(start_node)
    print(start_node, end = ' ')
    for combine_node in graph[start_node]:
        if combine_node not in nodes_searched:
            DFS_recursive(graph, combine_node)

In [6]:
'''
非递归版本
'''
# 利用栈实现
# 从起始结点把结点放入栈，然后弹出
# 每次弹出一个结点，就把该结点没有进过栈的邻结点放入栈，直到栈变空
def DFS_stack(graph, start_node):
    nodes_searched = set()
    stack = []  # 用List的pop和append模拟栈操作
    stack.append(start_node)
    nodes_searched.add(start_node)
    print(start_node, end = ' ')  # 先操作起始结点（打印）
    while len(stack) > 0:  # 用列表长度大于0表示集合非空
        cur = stack.pop()  # 弹栈
        for combine_node in graph[cur]:  # 判断弹栈元素的相邻结点是否在nodes_searched
            if combine_node not in nodes_searched:
                stack.append(cur)  # 先再次弹入刚弹出的元素！！！！！！！！！！！关键步骤！！！！！！
                stack.append(combine_node)  # 接着弹入最相邻的结点
                print(combine_node, end = ' ')  # 操作最相邻结点
                nodes_searched.add(combine_node)  # 将最相邻结点加入nodes_searched
                break  # 当有相邻结点进栈时，停止进栈，寻找该结点的相邻结点！！！！！！！深度优先的核心步骤！！！！！！！！！！

### 测试

In [7]:
GRAPH = {
    'A': ['B', 'F'],
    'B': ['C', 'I', 'G'],
    'C': ['B', 'I', 'D'],
    'D': ['C', 'I', 'G', 'H', 'E'],
    'E': ['D', 'H', 'F'],
    'F': ['A', 'G', 'E'],
    'G': ['B', 'F', 'H', 'D'],
    'H': ['G', 'D', 'E'],
    'I': ['B', 'C', 'D'],
}
BFS(GRAPH, "A")
print('')
DFS_recursive(GRAPH, "A")
print('')
DFS_stack(GRAPH, "A")

A B F C I G E D H 
A B C I D G F E H 
A B C I D G F E H 

# 最小生成树
----
**引例：假设要为一个镇的9个村庄假设通信网络做设计（图的每个结点相当于一个村庄，连接各个村庄需要花费的成本为各条边的权值），问要求该项目的成本最低，该如何设计路线？**

----
**定义：包含图的N个顶点和N-1条边的连通子图称为生成树**
****
**Prime法和Kruskal法的比较：Prime法是针对结点展开的，对于稠密图的情况好一些，Kruskal法是针对边展开的，边数少的时候效率会非常高，所以对于稀疏图有很大优势。**

## 1. 普里姆（Prim）法
----
**核心思路：任意选一个结点放入U集合，将剩下的结点放入V集合，寻找连接U和V的最短路径，将V中的该结点放入U中，如此往复，每次都找最近的边，直到V为空。（贪心算法）**
****
**算法分析：时间复杂度O（${n}^{2}$）,空间复杂度O(n)**  
**使用优先队列获取lowCost中的最小值可以将时间复杂度减少到O(nlogn)**

In [39]:
INF = float('inf')

def MiniSpanTree_Prim(graph):
    U = [0]  # U存放已连接结点
    V = list(range(1, len(graph)))  # V战队，存放未连接结点
    tree = []  # Tree记录成本
    lowCost = graph[0] # 记录U和V中各点连线距离
    while(len(V) > 0):
        '''
        在V中寻找离U最近的结点的下标，记作min
        '''
        min = 0  
        temp = INF
        for i in range(len(lowCost)):
            if (i not in U) and  (lowCost[i] < temp):
                min = i
                temp = lowCost[i]
        if lowCost[min] == 0:return False  # 说明列表中只含有0，无解
        '''
        将该结点从V中移出，放入U中
        '''
        U.append(min)
        V.remove(min)
        '''
        修改lowCost的值，lowCost记录的是上一个U和V中各点连线的距离，只需比较新加入点和V中各点的距离，取较小值替换lowCost中值即可
        '''
        tree.append(lowCost[min])
        for i in range(len(graph)):
            if graph[min][i] < lowCost[i] and i not in U:
                lowCost[i] = graph[min][i]
    return tree

In [40]:
G = [[0, 4, 1, 1],[4, 0, INF, 3],[1, INF, 0, 6],[1, 3, 6, 0]]

In [41]:
MiniSpanTree_Prim(G)

[1, 1, 3]

## 2. 克鲁斯卡尔（Kruskal）法
----
**核心思路：首先将N个结点标记为N类，并将边按权值从小到大排序。每次选择最短边，并将最短边的两个结点归为一类，如此循环，如过两个结点已为一类，则说明该边会形成环路，则跳过，直至边数达到N - 1（生成树定义边数为N -1）**  
****
**算法分析：对边排序的时间复杂度为O(eloge),合并结点类别的是时间复杂度为O（${n}^{2}$），空间复杂度O(n)**  
**使用并查集可以把合并结点类别的时间复杂度降为O(elogn)**

In [86]:
INF = float('inf')
def MiniSpanTree_Kruskal(graph):
    tree = []  # Tree记录成本
    length = len(graph)
    edges = []  # edges存放Edge类对象，记录每条边的权值和两个结点
    index_edges = 0  # edges列表的下表索引，从0开始
    Number = [i for i in range(length)]  # Number用于给每个结点编号，初始时每个结点编号等于自身结点号（哈希表思想，Number索引代表结点）
    '''
    将各条边用Edge对象表示，并按权值排序后存入edges数组
    '''
    for i in range(length - 1):
        for j in  range(i + 1, length):
            edges.append(Edge(graph[i][j], i, j))
    edges.sort(key = lambda x : x.weight)
    '''
    循环，每次将最短边的两个结点归为同一编号，如果两个结点已经为同一编号，说明会形成回路，则忽略，直至边长达到最小生成树定义（N - 1）
    '''
    while len(tree) < length - 1:
        min_i, min_j = edges[index_edges].node1, edges[index_edges].node2
        if Number[min_i] != Number[min_j]:
            tree.append(edges[index_edges].weight)
            '''
            如果两结点编号不同，则遍历Number数组，将所有编号为两结点编号的结点归同一编号
            '''
            for i in range(len(Number)):
                if Number[i] == Number[min_j]:
                    Number[i] = Number[min_i];
        index_edges += 1
    return tree
'''
生成一个Edge类，用于存放每条边的权值和相邻两个结点
'''
class Edge():
    def __init__(self, weight, node1, node2):
        self.weight = weight
        self.node1 = node1
        self.node2 = node2

In [88]:
G = [[0, 4, 1, 1],[4, 0, INF, 3],[1, INF, 0, 6],[1, 3, 6, 0]]

In [89]:
MiniSpanTree_Kruskal(G)

[1, 1, 3]

# 最短路径
****
**引例：在北上广这类大城市，考虑从A地到B地，如何换乘到达，使得进过的路径最短？如果要求任意城市里任意两地的最短路径呢？**

## 1. 迪杰斯特拉（Dijkstra）法
****
**核心思想：将起始结点放入集合U，将剩下结点放入集合V，初始化shortestPath数组为起始节点至各结点的路径。每次循环将shortestPath中的最短路径结点加入U，并比较借助该节点的相邻结点的最短路径是否变小，如是，则更新shortestPath。**
****
**算法分析：时间复杂度O(${n}^{2}$)，如果计算任意两两结点间的最短路径，则加一层循环，时间复杂度变为O(${n}^{3}$)**

In [7]:
INF = float('inf')
def ShortestPath_Dijkstra(graph, start):
    '''
    初始化定义U、V、前驱、最短路径数组
    '''
    length = len(graph)
    U = [start]
    V = [i for i in range(length) if i != start]
    forward = [0] * length  # 前驱，用于记录最短路径的连接方式
    shortestPath = graph[start]  # 最短路径，用于记录最短路径的值
    while len(V) > 0:
        '''
        查找最短路径的最小值结点
        '''
        temp = INF
        min = start
        for i in range(length):
            if (i not in U) and (shortestPath[i] < temp):
                temp = shortestPath[i]
                min = i
        if min == start:return False
        '''
        将最小值结点从V中放入U中
        '''
        U.append(min)
        V.remove(min)
        '''
        观察min结点的相邻结点通过min的路径是否小于最短路径，如是，则更新shortestPath
        '''
        for i in range(length):
            if (i not in U) and (0 < graph[min][i] < INF) and (graph[min][i] + shortestPath[min] < shortestPath[i]):
                shortestPath[i] = graph[min][i] + shortestPath[min]
                forward[i] = min
    return shortestPath,forward

In [5]:
G = [[0,2,4,INF,INF,1],[2,0,2,INF,3,INF],[4,2,0,1,3,5],[INF,INF,1,0,4,5],[INF,3,3,4,0,INF],[1,INF,5,5,INF,0]]

In [6]:
ShortestPath_Dijkstra(G,0)

([0, 2, 4, 5, 5, 1], [0, 0, 0, 2, 1, 0])

## 2. 佛洛依德（Floyd）算法
****
**核心思想：初始化两个矩阵，一个是最短路径矩阵，一个是前驱矩阵。三层循环粗暴遍历：如果start->end的路径 ＞ start -> mid -> end 的路径，则更新最短路径矩阵和前驱矩阵。**
****
**算法分析：时间复杂度O(${n}^{3}$)，但十分巧妙美观，适用于需求为所有顶点至所有顶点的问题**

In [73]:
def ShortestPath_Floyd(graph, begin, final):
    length = len(graph)
    shortestPath = graph
    forword = [list(range(length)) for i in  range(length)]
    for start in range(length):
        for end in range(length):
            for mid in range(length):
                if shortestPath[start][end] > shortestPath[start][mid] + shortestPath[mid][end]:
                    shortestPath[start][end] = shortestPath[start][mid] + shortestPath[mid][end]
                    forword[start][end] = forword[start][mid]
#     return shortestPath,forword
    print('最短路径长度：', shortestPath[begin][final])
    print('路径：', end = '')
    key = forword[begin][final]
    print(begin, end = '->')
    while key != final:
        print(key, end = '->')
        key = forword[forword[begin][final]][final]
    print(key)

In [74]:
G = [[0,2,4,INF,INF,1],[2,0,2,INF,3,INF],[4,2,0,1,3,5],[INF,INF,1,0,4,5],[INF,3,3,4,0,INF],[1,INF,5,5,INF,0]]

In [72]:
ShortestPath_Floyd(G,0,3)

最短路径长度： 5
路径：0->2->3


# 拓扑排序
****
**有向无环图（DAG，Driected Acyclic Graph）的所有顶点的线性序列，满足：**
1. 每个顶点只出现一次
2. 若存在一条从A->B的路径，则序列中A必须在B前面
****
**实际应用：用来排序有依赖关系的任务，如工程项目、软件开发、教学安排、电影制作等等项目，一定是无环的有向图**
****
**核心思想：从DAG中选择一个入度为0的顶点输出，并删去此顶点及其连接的弧，重复此步骤直到图为空（成功）或图中不存在入度为0的结点（失败，有环）**
****
**算法分析：将初始入度为0的结点置入队列的时间复杂度为O(n)，在队列中进行操作的时间复杂度为O(e)，总计O(n + e)**

In [115]:
from queue import Queue
def TopologicalSort(graph):
    count = 0
    inDegree = [0] * len(graph)
    for key in graph:
        for i in graph[key]:
            inDegree[i] += 1
    queue = Queue()  # 用于存放入度为0的顶点，避免每个查找都要去遍历入度表查找有没有入度为0的顶点
    for i in range(len(inDegree)):
        if inDegree[i] == 0:
            queue.put(i)
    while queue:
        temp = queue.get()
        print(temp, end = ' ')
        count += 1
        for i in graph[temp]:
            inDegree[i] -= 1
            if inDegree[i] == 0:
                queue.put(i)
    if count != len(graph):return False
    else:return True

In [116]:
GRAPH = {
    0: [],
    1: [],
    2: [3],
    3: [1],
    4: [0, 1],
    5: [0, 2]
}

In [ ]:
TopologicalSort(GRAPH)

4 5 0 2 3 1 